In [1]:
import pandas as pd
import numpy as np
import csv
import string

In [2]:
train_data=pd.read_csv('olid-training-v1.0.tsv', delimiter='\t', encoding='utf-8')

train_tweets = train_data[['tweet']] #Extract tweets
train_label_a= train_data['subtask_a'].tolist() #Extract subtsak_a labels
train_label_b= train_data['subtask_b'].tolist() #Extract subtsak_b labels
train_label_c= train_data['subtask_c'].tolist() #Extract subtsak_c labels

In [3]:
#Function to clean tweets in a data frame's tweet column
def clean_tweets(df):
    
    punctuations = string.punctuation
    
    df.loc[:, 'tweet'] = df.tweet.str.replace('@USER', '') #Remove mentions (@USER)
    df.loc[:, 'tweet'] = df.tweet.str.replace('URL', '') #Remove URLs
    df.loc[:, 'tweet'] = df.tweet.str.replace('&amp', 'and') #Replace ampersand (&) with and
    df.loc[:, 'tweet'] = df.tweet.str.replace('&lt','') #Remove &lt
    df.loc[:, 'tweet'] = df.tweet.str.replace('&gt','') #Remove &gt
    df.loc[:, 'tweet'] = df.tweet.str.replace('\d+','') #Remove numbers

    #Remove punctuations
    for punctuation in punctuations:
        df.loc[:, 'tweet'] = df.tweet.str.replace(punctuation, '')

    df.loc[:, 'tweet'] = df.astype(str).apply(
        lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii')
    ) #Remove emojis
    df.loc[:, 'tweet'] = df.tweet.str.strip() #Trim leading and trailing whitespaces

In [4]:
clean_tweets(train_tweets)

C:\Users\Joel\Anaconda3\envs\Pandas\lib\site-packages\pandas\core\indexing.py:576: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [5]:
train_tweets_a = train_tweets['tweet'].tolist()

In [13]:
#Remove stop words
import nltk
# nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

filtered_train_tweets_a = []

for text in train_tweets_a:
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
    filtered_tweet = (" ").join(tokens_without_sw)
    filtered_train_tweets_a.append(filtered_tweet)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Joel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
len(filtered_train_tweets_a)

13240

### Preparing the test sets

In [15]:
#Read tweets from test sets
test_tweets_a=pd.read_csv('testset-levela.tsv', delimiter='\t', encoding='utf-8', usecols=['tweet'])
test_tweets_b=pd.read_csv('testset-levelb.tsv', delimiter='\t', encoding='utf-8', usecols=['tweet'])
test_tweets_c=pd.read_csv('testset-levelc.tsv', delimiter='\t', encoding='utf-8', usecols=['tweet'])

#Read tweet labels
test_label_a=pd.read_csv('labels-levela.csv', encoding='utf-8', 
                         index_col=False, names=['id', 'class_a'], usecols=['class_a'])
test_label_b=pd.read_csv('labels-levelb.csv', encoding='utf-8', 
                         index_col=False, names=['id', 'class_b'], usecols=['class_b'])
test_label_c=pd.read_csv('labels-levelc.csv', encoding='utf-8', 
                         index_col=False, names=['id', 'class_c'], usecols=['class_c'])

test_label_a=test_label_a['class_a'].tolist()
test_label_b=test_label_b['class_b'].tolist()
test_label_c=test_label_c['class_c'].tolist()

#Clean tweets in test sets
clean_tweets(test_tweets_a)
clean_tweets(test_tweets_b)
clean_tweets(test_tweets_c)

test_tweets_a = test_tweets_a['tweet'].tolist()
test_tweets_b = test_tweets_b['tweet'].tolist()
test_tweets_c = test_tweets_c['tweet'].tolist()

In [23]:
tweets = train_tweets_a + test_tweets_a
labels = train_label_a + test_label_a

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

features = vectorizer.fit_transform(tweets)
features_nd = features.toarray()

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    features_nd,
    labels,
    train_size=0.9390070921985816,
    shuffle=False
)

In [26]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression(max_iter=1000)

In [27]:
log_model = log_model.fit(X=X_train, y=y_train)

In [28]:
y_pred = log_model.predict(X_test)

In [29]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, y_pred))

0.7965116279069767
